# How do these courses relate?

A large number of the coures that are offered within this training program are related to one another and combine together to create "Pathways". The intention for these pathways is for the courses to provide expertise in a given area that together enable you to take on larger proejcts. The interactive networks below will give additional details about a course if clicked upon, and you are able to move the network around the holding left click and dragging. 

In [1]:
from pyvis.network import Network
import pandas as pd
import networkx as nx

# Load the data
file_path = '../data/admin/workshop_details.csv'  # Adjust the path to your actual file location
workshop_data = pd.read_csv(file_path)

# Fill NaN in pre_reqs and handle empty pathways
workshop_data.fillna({'pre_reqs': '', 'pathways': 'None'}, inplace=True)

# Extract unique pathways and create color mapping
pathways = set()
workshop_data['pathways'].str.split(',').apply(pathways.update)
pathways.discard('None')  # Remove 'None' if no specific pathway is assigned
colors = ['#FFD700', '#FF6347', '#4682B4', '#32CD32', '#FFA500', '#6A5ACD', '#FF4500', '#2E8B57']
color_map = {pathway: colors[i % len(colors)] for i, pathway in enumerate(pathways)}

# Create a network for each pathway
for pathway in pathways:
    # Filter workshops for the current pathway
    pathway_data = workshop_data[workshop_data['pathways'].str.contains(pathway)]
    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes and edges with levels
    for _, row in pathway_data.iterrows():
        color = color_map[pathway]
        level = row['course_level']
        G.add_node(row['workshop_name'], title='', color=color, level=level)  # Add level attribute
        prereqs = row['pre_reqs'].split(',')
        for prereq in prereqs:
            prereq = prereq.strip()
            if prereq:
                G.add_edge(prereq, row['workshop_name'])

    # Now create detailed titles for each node
    for node in G.nodes:
        prereqs = [edge[0] for edge in G.in_edges(node)]
        subsequent_courses = [edge[1] for edge in G.out_edges(node)]
        prereqs_text = ', '.join(prereqs) if prereqs else 'None'
        subsequent_courses_text = ', '.join(subsequent_courses) if subsequent_courses else 'None'
        title_text = (f"Course Name: {node}\n"
                      f"Course Pre-reqs: {prereqs_text}\n"
                      f"Subsequent Courses: {subsequent_courses_text}")
        G.nodes[node]['title'] = title_text

    # Initialize PyVis network
    nt = Network(notebook=True, directed=True, cdn_resources='remote')
    nt.from_nx(G)
    
    # Set hierarchical layout options
    hierarchical_options = """
    {
      "layout": {
        "hierarchical": {
          "enabled": true,
          "levelSeparation": 150,
          "nodeSpacing": 250,
          "treeSpacing": 300,
          "blockShifting": true,
          "edgeMinimization": true,
          "parentCentralization": true,
          "direction": "UD",        
          "sortMethod": "hubsize"   
        }
      },
      "interaction": {
        "navigationButtons": true,
        "keyboard": true
      },
      "physics": {
        "enabled": false
      }
    }
    """
    nt.set_options(hierarchical_options)

    # Option to add all of the buttons for navigation to the visualization, this can't be used in conjunction with setting 
    # individual options above and so needs to be commented out to experiment. 
    # nt.show_buttons()

    # Save and display the network
    output_file = f"../_static/workshops_network_{pathway.replace(' ', '_').lower()}.html"  # File name based on pathway
    nt.show(output_file)


../_static/workshops_network_r_ds.html
../_static/workshops_network_python_ds.html
../_static/workshops_network_hpc.html


In [2]:
from IPython.display import HTML, display

# Assuming you've already saved each network to an HTML file
for pathway in pathways:
    file_name = f"../_static/workshops_network_{pathway.replace(' ', '_').lower()}.html"
    
    # Generate a unique ID for each iframe
    iframe_id = f"iframe-{pathway.replace(' ', '_').lower()}"

    # Create a display HTML block with a title
    html_str = f"<div style='text-align: center;'><h3>{pathway} Pathway</h3></div>"
    html_str += f"<iframe id='{iframe_id}' src='{file_name}' width='100%' height='750px' style='border:none;'></iframe>"
    
    # Include JavaScript to adjust the iframe height
    js_code = f"""
    <script>
    function adjustIframeHeight_{iframe_id}() {{
        const iframe = document.getElementById('{iframe_id}');
        if (iframe) {{
            iframe.style.height = iframe.offsetWidth + 'px';
        }}
    }}

    // Adjust height on load and resize
    window.onload = adjustIframeHeight_{iframe_id};
    window.onresize = adjustIframeHeight_{iframe_id};
    </script>
    """
    
    html_str += js_code
    display(HTML(html_str))